In [1]:

# ^^^ pyforest auto-imports - don't write above this line


# Imports

## Libraries

In [2]:
import unidecode
import wikipedia
import warnings
warnings.filterwarnings('ignore')
from functions import *
import pickle
import pandas as pd

## Demonym Dictionary

In [3]:
with open("../my_saved_data/demonym_dictionary.pickle", "rb+") as f:
    demonym_dictionary = pickle.load(f)

# Adding City (Restaurant Inspection) Data
cities with most mexican immigrants in us: https://247wallst.com/economy/2017/01/27/us-cities-with-the-most-mexican-immigrants/

## NYC

In [4]:
rest_insp = pd.read_csv("../food_inspections_data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv")
mex_rest_insp = rest_insp[rest_insp['CUISINE DESCRIPTION'] == 'Mexican']

### Removing duplicate restaurants from nyc mexican restaurants df

In [5]:
unique_nyc_mex = mex_rest_insp.drop_duplicates(subset=['DBA'], keep='first')
unique_nyc_mex.reset_index(inplace=True, drop=True)
unique_nyc_mex.shape

(849, 26)

In [6]:
# list of restaurant names to compare to values in demonym_dictionary
nyc_rest_insp_names = [x.lower() for x in unique_nyc_mex['DBA']]

### Getting matches 

In [7]:
rest_matches_nyc = find_matches(nyc_rest_insp_names, demonym_dictionary)

In [8]:
len(rest_matches_nyc)

188

In [9]:
unique_rest_matches_nyc = list(set(rest_matches_nyc))
# was 44 before additions to demonym dict ... was 169 before word corrections
len(unique_rest_matches_nyc)

88

In [10]:
nyc_rest_w_region_matches = find_regions_for_restaurants(rest_matches_nyc, demonym_dictionary)

In [47]:
# add to big df
final_df_nyc = add_city_to_all_cities_dict(nyc_rest_w_region_matches, "NYC")

## Chicago 

In [14]:
chi_data = pd.read_csv("../food_inspections_data/Food_Inspections_chicago.csv", )
chi_data.shape

(206916, 17)

In [15]:
chi_rest_deli = chi_data[chi_data['Facility Type'].isin(['Restaurant', 'DELI/GROCERY', 
                                                        'Mobile Food Preparer', 'Bakery',
                                                        'Tavern', 'Tavern/Bar', 'Mobile Frozen Desserts Vendor',
                                                        'Mobile Food Dispenser', 'JUICE BAR/GROCERY',
                                                        'GROCERY STORE /PHARMACY', 'COMMISARY RESTAURANT',
                                                        'GROCERY/DELI', 'FOOD BOOTH', 'RESTAURANT/BAR',
                                                        'MOBILE DESSERTS VENDOR', 'GROCERY(GAS STATION)',
                                                        'CANDY/GELATO', 'COFFEE SHOP',
                                                        'Grocery & Restaurant', 'CANDY/GELATO', 'GROCERY STORE/BAKERY'
                                                        'GROCERY/RESTAURANT', 'CANDY SHOP', 'REST/GROCERY',
                                                        'cafeteria', 'DINING HALL', 'Deli', 'DELI/GROCERY STORE',
                                                        'GROCERY& RESTAURANT', 'Theater & Restaurant',
                                                        'RESTAURANT/GROCERY STORE', 'Banquet/kitchen',
                                                        'CONVENIENCE', 'Cafeteria', 'MOBILE FROZEN DESSERT VENDOR', 
                                                         'SLAUGHTER HOUSE/ GROCERY','MOBILE FOOD TRUCK',
                                                        'Mobile frozen dessert vendor', 'convenience/drug store',
                                                         'MOBILE FOOD DESSERTS VENDOR', 'RESTAURANT/HOSPITAL',
                                                         'MOBILE FROZEN DESSERTS VENDOR', 'RESTAURANT/BAKERY',
                                                        'CAFETERIA', 'BAR/GRILL', 'PALETERIA', 'GROCERY AND BUTCHER',
                                                        'TAVERN GRILL', 'GAS STATION /GROCERY',
                                                        'GROCERY/TAQUERIA', 'grocery & restaurant', 
                                                        'BAKERY/DELI', 'GROCERY/CAFE', 'TENT RSTAURANT', 
                                                        'RESTAURANT/LIQUOR', 'GROCERY & RESTAURANT', 'tavern/restaurant',
                                                        'DELI/BAKERY', 'GROCERY/BAKERY', 'RESTAURANT/BAR/THEATER',
                                                        'RESTAURANT AND LIQUOR', 'GROCERY STORE/ RESTAURANT', 
                                                        'BAKERY/GROCERY', 'TAVERN/RESTAURANT', 'GROCERY STORE/DELI', 
                                                        'BAKERY/ RESTAURANT', 'bakery/restaurant', ])]
chi_rest_deli.shape

(142995, 17)

### Getting Names of restaurants

In [16]:
print(chi_rest_deli['DBA Name'].isnull().values.any())

chi_biz_names = [x.lower() for x in chi_rest_deli['DBA Name']]
len(chi_biz_names)

False


142995

In [17]:
unique_chi_biz_names = list(set(chi_biz_names))
len(unique_chi_biz_names)

15166

### Getting Matches with Dem Dict

In [18]:
chi_match_list = find_matches(unique_chi_biz_names, demonym_dictionary)
len(chi_match_list)

492

In [19]:
unique_rest_matches_chi = list(set(chi_match_list))
len(unique_rest_matches_chi)

284

In [20]:
chi_rest_w_region_matches = find_regions_for_restaurants(unique_rest_matches_chi, demonym_dictionary)

In [21]:
only_1_match_counter = 0
for key, value in chi_rest_w_region_matches.items():
    for key1, value1 in value.items():
        if len(value1) == 1:
            only_1_match_counter += 1

In [22]:
only_1_match_counter

211

In [23]:
len(list(chi_rest_w_region_matches.keys()))

284

In [48]:
final_df_chi = add_city_to_all_cities_dict(chi_rest_w_region_matches, "Chi")

## Dallas

In [25]:
dallas_data = pd.read_csv("../food_inspections_data/Dallas_inspections_October_2016_to_Present_.csv")
dallas_data.shape

(45844, 114)

### Only unique dallas restaurants

In [26]:
unique_dallas_data = dallas_data.drop_duplicates(subset=['Restaurant Name'], keep='first')
unique_dallas_data.shape

(7474, 114)

### Removing columns from data

In [27]:
# most of the columns relate to inspections
col_names = list(unique_dallas_data.columns)

In [28]:
col_names[-3:]
unique_dallas_data.drop(columns = [x for x in col_names[11:-4]], inplace=True)

In [29]:
# drop nulls from restaurant names and get names 
dallas_rest_list = [x.lower() for x in unique_dallas_data['Restaurant Name'] if type(x) == str]

In [30]:
len(dallas_rest_list)

7473

### Getting matches to dem dict

In [31]:
dal_match_list = find_matches(dallas_rest_list, demonym_dictionary)
len(dal_match_list)

186

In [32]:
unique_rest_matches_dal = list(set(dal_match_list))
len(unique_rest_matches_dal)

117

In [33]:
dal_rest_w_region_matches = find_regions_for_restaurants(unique_rest_matches_dal, demonym_dictionary)

In [49]:
final_df_dal = add_city_to_all_cities_dict(dal_rest_w_region_matches, "Dal")

## San Francisco 

In [36]:
sf_data = pd.read_csv("../food_inspections_data/san_fran_Restaurant_Scores_-_LIVES_Standard.csv")
sf_data.shape

(53973, 23)

In [37]:
sf_data.head(2)

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,inspection_type,violation_id,violation_description,risk_category,Neighborhoods (old),Police Districts,Supervisor Districts,Fire Prevention Districts,Zip Codes,Analysis Neighborhoods
0,69618,Fancy Wheatfield Bakery,1362 Stockton St,San Francisco,CA,94133,NaN,NaN,NaN,NaN,...,Complaint,69618_20190304_103130,Inadequate sewage or wastewater disposal,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
1,97975,BREADBELLY,1408 Clement St,San Francisco,CA,94118,NaN,NaN,NaN,1.415724e+10,...,Routine - Unscheduled,97975_20190725_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
sf_data_unique = sf_data.drop_duplicates(subset=['business_name'], keep='first')
sf_data_unique.shape

(5775, 23)

In [39]:
lower_biz_name = [x.lower() for x in sf_data['business_name']]
# this is slightly smaller than the set of unique business names without lowering all of the characters (by 1000)
unique_sf_biz_names = list(set(lower_biz_name))
len(set(unique_sf_biz_names)) 

5672

### Getting Names of Restaurants

In [40]:
sf_match_list = find_matches(unique_sf_biz_names, demonym_dictionary)
len(sf_match_list)

45

In [41]:
unique_rest_matches_sf = list(set(sf_match_list))
len(unique_rest_matches_sf)

17

In [42]:
sf_rest_w_region_matches = find_regions_for_restaurants(unique_rest_matches_sf, demonym_dictionary)

In [50]:
final_df_sf = add_city_to_all_cities_dict(sf_rest_w_region_matches, "SF")

# Combining All Restaurant Matches into One df

In [51]:
# city_df_columns = ['Restaurant', 'City', 'Matched_word', 'Region', 'Multiple_regions_flag', "Final_region"]
final_df = pd.concat([final_df_nyc, final_df_chi, final_df_dal, final_df_sf])

## Saving All Restaurants with their Matches

In [52]:
final_df.to_csv("../my_saved_data/All_restaurant_matches_df.csv")